In [1]:
import torch
import numpy as np

In [2]:
seqs = []
refs = []
scores = []
for f in range(1, 4):
    scores1 = np.load(f"/home/ali.lawati/mol-incontext/input/embed/mmcl_attr-chebi-{f}-epochs300-loop.mistral-7B.scores.npy")
    print(len(scores1))
    print(scores1[7], scores1[9], scores1[18], scores1[19], scores1[21], scores1[29], scores1[66])
    embeds1 = np.load(f"/home/ali.lawati/mol-incontext/input/embed/mmcl_attr-chebi-{f}-epochs300-embeds.npz")
    seqs1 = torch.tensor(embeds1['embeds']) #.flip(dims=[1])] #torch.cat([torch.zeros(3297, 5-f, 64), torch.tensor(embeds1['embeds']).flip(dims=[1])], dim=1)
    #print(embeds1['embeds'][7])
    refs1 = np.reshape(embeds1['test_pool'], (embeds1['test_pool'].shape[0],1,-1))
    seqs.append(seqs1)
    refs.append(torch.tensor(refs1))
    scores.append(scores1)

#seqs_tensor = torch.cat(seqs)
#scores_tensor = torch.tensor(np.concatenate(scores))
#refs_tensor = torch.tensor(np.concatenate(refs))

l_scores_tensor = torch.tensor(scores[-1])

3297
0.16574624158406065 0.3114123715955396 0.8198593291495693 0.8547645909981955 0.24998919092138122 0.970237533262574 0.5951163207378447
3297
0.16574624158406065 0.9113480848578538 0.8198593291495693 0.8547645909981955 0.8268080505663535 0.970237533262574 0.5951163207378447
3297
0.200726550812963 0.9113480848578538 0.8198593291495693 0.7605280565989978 0.8268080505663535 0.970237533262574 0.859036534402326


In [7]:
idx = np.where((scores[2] > scores[1]) & (scores[2] > scores[0]) & (scores[2] > .7))[0]
dmb = np.where((scores[2] < scores[1]) & (scores[1] < scores[0]) & (scores[0] < .5))[0]
len(dmb)

#create_expert_trajs(refs[2][dmb], seqs[2][dmb])

85

In [8]:
seqs[2][idx].shape
refs[2][idx].shape

refs[2][idx[0]]+seqs[2][idx[0]]

tensor([[ 1.6021e-02, -2.3202e-02,  4.2733e-02,  1.6160e-01, -9.1340e-05,
          9.8902e-02, -4.4263e-02, -1.5083e-01,  1.2593e-02, -3.1048e-02,
         -1.2606e-01,  1.1833e-01,  1.0150e-01,  2.4165e-03,  2.6916e-02,
         -2.0499e-02, -7.3339e-03,  3.7603e-02,  3.6223e-02, -3.8263e-02,
          6.1981e-02, -3.7109e-02,  6.2713e-03,  1.0167e-01,  9.5623e-02,
         -1.3048e-01, -4.3612e-02, -1.1486e-01, -5.3162e-02, -7.1722e-02,
          9.7927e-02, -4.6405e-02, -2.3655e-01,  1.1613e-01, -3.3322e-02,
          7.2241e-02,  1.9261e-02, -4.4375e-03,  6.7154e-02,  1.3967e-01,
         -8.5062e-02, -1.8700e-02, -4.7262e-02,  9.1540e-02, -1.1362e-01,
          6.5768e-02,  5.5655e-02, -4.9894e-02,  3.6377e-02,  8.4522e-03,
          3.1113e-02, -7.9134e-02,  1.0564e-01,  9.7155e-02,  1.6680e-01,
          1.1739e-01,  6.5591e-03, -2.8080e-02,  1.0765e-01, -1.1404e-01,
         -1.5652e-02, -2.9581e-02,  9.4983e-02, -1.4238e-01],
        [ 1.8196e-02, -3.0360e-02,  3.9709e-02,  1

In [12]:
def create_expert_states(refs, seqs):
    combined = torch.cat([refs, seqs], dim=1)  # Shape: [batch, L, dim]
    cumulative_sum = torch.cumsum(combined, dim=1)  # Shape: [batch, L, dim]
    return cumulative_sum

def create_triplets(states, actions):
    next_states = states[:, 1:, :]  # Shape: [BATCH, 3, 64]
    current_states = states[:, :3, :]  # Shape: [BATCH, 3, 64]
    #triplets = torch.stack([current_states, actions, next_states], dim=2)
    #return triplets
    return current_states, actions, next_states
actions = seqs[2][idx]
states = create_expert_states(refs[2][idx], seqs[2][idx])
expert_trajs = create_triplets(states, actions)

                               


#print(seqs[2][idx][0])
a = create_triplets(states, actions)
print(a[0].shape, a[1].shape, a[2].shape)


torch.Size([129, 3, 64]) torch.Size([129, 3, 64]) torch.Size([129, 3, 64])


In [23]:
torch.ones(actions.shape[0]).shape

torch.Size([129])

In [100]:
import random
def create_policy_traj(states, actions, B=32, ML=3):
    actions = actions.reshape(-1 ,64)
    states = states.reshape(-1 ,64)
    
    actions_B = actions.unsqueeze(0).expand(B, -1, 64)
    states_B = states.unsqueeze(0).expand(B, -1, 64)

    sample_sizes = torch.randint(1, ML, (B,))
    mask = (torch.arange(ML-1).expand(B, -1) < sample_sizes.unsqueeze(1))
    probabilities = torch.ones(B, actions.shape[0])

    indices = torch.multinomial(probabilities, ML-1, replacement=False)
    actions_s = actions_B[torch.arange(32).unsqueeze(1), indices]
    result = actions_s*mask.unsqueeze(-1).float()
    
    action_add = result.sum(dim=1)
    indices = torch.multinomial(torch.ones(B, actions.shape[0]), 1, replacement=False)
    p_action = actions_B[torch.arange(32).unsqueeze(1), indices].squeeze(1)
    indices = torch.multinomial(torch.ones(B, states.shape[0]), 1, replacement=False)
    p_state = states_B[torch.arange(32).unsqueeze(1), indices].squeeze(1)
    
    p_state = p_state + action_add
    p_next_state = p_action + p_state

    return p_state, p_action, p_next_state

x = create_policy_traj(refs[2], seqs[2])
print(x[0].shape, x[1].shape, x[2].shape)

torch.Size([32, 64])
p_action.shape torch.Size([32, 64])
torch.Size([32, 1]) torch.Size([32, 3297, 64])
p_state.shape torch.Size([32, 64])
torch.Size([32, 64]) torch.Size([32, 64]) torch.Size([32, 64])


In [16]:
sample_sizes = torch.randint(1, 4, (64,))

tensor([2, 2, 1, 1, 2, 3, 2, 3, 3, 2, 3, 3, 3, 3, 1, 3, 3, 2, 3, 2, 3, 3, 1, 3,
        2, 2, 3, 1, 1, 2, 3, 3, 3, 3, 3, 1, 1, 1, 1, 2, 3, 1, 3, 2, 2, 2, 2, 2,
        1, 1, 2, 3, 2, 3, 2, 2, 2, 3, 3, 3, 1, 2, 2, 3])

In [60]:
B = 32

actions = actions.reshape(-1 ,64)
print(actions.shape)
sample_sizes = torch.randint(1, 4, (B,))
mask = (torch.arange(3).expand(B, -1) < sample_sizes.unsqueeze(1))
probabilities = torch.ones(B, actions.shape[0])
indices = torch.multinomial(probabilities, 3, replacement=False)
print('indices', indices.shape)
print('actions', actions.shape)
print('mask', mask.shape)
actions_B = actions.unsqueeze(0).expand(B, -1, 64)
print(actions_B.shape)
actions_s = actions_B[torch.arange(32).unsqueeze(1), indices]
print(actions_s.shape)
result = actions_s*mask.unsqueeze(-1).float()
print(result.shape)
print(result.sum(dim=1).shape)


torch.Size([387, 64])
indices torch.Size([32, 3])
actions torch.Size([387, 64])
mask torch.Size([32, 3])
torch.Size([32, 387, 64])
torch.Size([32, 3, 64])


RuntimeError: The size of tensor a (3) must match the size of tensor b (32) at non-singleton dimension 1

In [22]:
 (torch.arange(3).expand(6, -1) < sample_sizes.unsqueeze(1))

tensor([[ True,  True, False],
        [ True,  True, False],
        [ True, False, False],
        [ True, False, False],
        [ True,  True, False],
        [ True,  True,  True],
        [ True,  True, False],
        [ True,  True,  True],
        [ True,  True,  True],
        [ True,  True, False],
        [ True,  True,  True],
        [ True,  True,  True],
        [ True,  True,  True],
        [ True,  True,  True],
        [ True, False, False],
        [ True,  True,  True],
        [ True,  True,  True],
        [ True,  True, False],
        [ True,  True,  True],
        [ True,  True, False],
        [ True,  True,  True],
        [ True,  True,  True],
        [ True, False, False],
        [ True,  True,  True],
        [ True,  True, False],
        [ True,  True, False],
        [ True,  True,  True],
        [ True, False, False],
        [ True, False, False],
        [ True,  True, False],
        [ True,  True,  True],
        [ True,  True,  True],
        